# Demo of Earthquake Detection at Cook Inlet

Author: Qibin Shi

Tech support: Yiyu Ni

In [ ]:
import sys
sys.path.append('../src/denoiser/')
sys.path.append('../src/ensemble_picker/')

import gc
import glob
import h5py
import pygmt
import numpy as np
import pandas as pd

import torch
import pyocto
import seisbench.models as sbm
from ELEP.elep.ensemble_coherence import ensemble_semblance
from ELEP.elep.trigger_func import picks_summary_simple

from datetime import datetime
from das_util import try_gpu
from detect_util import *
from joblib import Parallel, delayed
from tqdm import tqdm

import obspy
from obspy import UTCDateTime, read_events
from obspy.clients.fdsn.client import Client

###
filepath = '/fd1/QibinShi_data/akdas/qibin_data/elep_pyocto/coast_only/'

## 1. Buidling catalog with Only Coastal Stations

#### Station availability

In [ ]:
client = Client("IRIS")
t1 = UTCDateTime("2023-12-01")
t2 = UTCDateTime("2023-12-31")

inventory = client.get_stations(network="AK,AV", channel="BH?",
                                starttime=t1, endtime=t2, 
                                maxlatitude=60.1809, minlatitude=58.5911, 
                                maxlongitude=-150.6555, minlongitude=-153.7177)

#### Detect 30 days

In [ ]:
### ELEP models
devcc = try_gpu(i=1)

pn_ethz_model = sbm.EQTransformer.from_pretrained("ethz")
pn_neic_model = sbm.EQTransformer.from_pretrained("neic")
pn_scedc_model = sbm.EQTransformer.from_pretrained("scedc")
pn_stead_model = sbm.EQTransformer.from_pretrained("stead")
pn_geofon_model = sbm.EQTransformer.from_pretrained("geofon")
pn_instance_model = sbm.EQTransformer.from_pretrained("instance")

pn_ethz_model.to(devcc)
pn_neic_model.to(devcc)
pn_scedc_model.to(devcc)
pn_stead_model.to(devcc)
pn_geofon_model.to(devcc)
pn_instance_model.to(devcc)

pn_ethz_model.eval()
pn_neic_model.eval()
pn_scedc_model.eval()
pn_stead_model.eval()
pn_geofon_model.eval()
pn_instance_model.eval()

list_models = [pn_ethz_model,
               pn_neic_model,
               pn_scedc_model,
               pn_stead_model,
               pn_geofon_model,
               pn_instance_model]

### Loop over days
for i in range(30):
    t1 = UTCDateTime("2023-12-07") + i * 86400
    ### loop over stations
    for net in inventory:
        network = net.code

        for sta in net:
            station = sta.code
            
            print(network, station, t1)  
            
            detect_on_fly(network, station, t1, filepath, 6000, 3000, list_models, devcc)

#### Prepare picks for association

In [ ]:
### Merge picks from all stations
csv_list = glob.glob(filepath+'1month/*2023*.csv')
all_csv =[]
for i in csv_list:
    all_csv.append(pd.read_csv(i, index_col=0))
df = pd.concat(all_csv, axis=0)

# change to Octo format
df.loc[df['trace_p_arrival'].notna(), 'phase'] = "P"
sta_p_time = df.loc[df['trace_p_arrival'].notna(), ['station_code','phase','trace_p_arrival']].rename(
    columns={"station_code": "station", "trace_p_arrival": "time"})

df.loc[df['trace_s_arrival'].notna(), 'phase'] = "S"
sta_s_time = df.loc[df['trace_s_arrival'].notna(), ['station_code','phase','trace_s_arrival']].rename(
    columns={"station_code": "station", "trace_s_arrival": "time"})

picks = pd.concat(objs = [sta_p_time,sta_s_time] , axis=0)
picks['time'] = picks['time'].apply(lambda x: (datetime.strptime(x,'%Y-%m-%dT%H:%M:%S.%fZ')).timestamp())

picks.to_csv(filepath + 'picks_octo.csv', index=False)
picks.head()

#### Prepare stations for association

In [ ]:
### Station table for PyOcto
# client = Client("IRIS")
# t1 = UTCDateTime("2023-12-01")
# t2 = UTCDateTime("2023-12-31")
# inventory = client.get_stations(network="AK,AV", channel="BH?",
#                                 starttime=t1, endtime=t2, 
#                                 maxlatitude=60.1809, minlatitude=58.5911, 
#                                 maxlongitude=-150.6555, minlongitude=-153.7177)
# stations_table = pd.DataFrame(columns=['id','longitude','latitude', 'elevation'])

# for net in inventory:
#     for sta in net:
#         station = sta.code
#         temp = pd.DataFrame(data={'id': sta.code,
#                                   'longitude': sta.longitude,
#                                   'latitude': sta.latitude, 
#                                   'elevation': sta.elevation},index=[0])
#         stations_table = pd.concat([stations_table,temp],ignore_index=True)

stations_table = pd.read_csv(filepath + 'stations_table.csv')

velocity_model = pyocto.VelocityModel0D(
    p_velocity=7.0,
    s_velocity=4.0,
    tolerance=2.0,
)
# associator = pyocto.OctoAssociator.from_area(
#     lat=(57, 61),
#     lon=(-155, -149),
#     zlim=(0, 200),
#     time_before=300,
#     velocity_model=velocity_model,
#     n_picks=6,
#     n_p_picks=3,
#     n_s_picks=3,
#     n_p_and_s_picks=3,
# )

# associator = pyocto.OctoAssociator.from_area(
#     lat=(57, 61),
#     lon=(-155, -149),
#     zlim=(0, 200),
#     time_before=300,
#     velocity_model=velocity_model,
#     n_picks=10,
#     n_p_picks=5,
#     n_s_picks=5,
#     n_p_and_s_picks=3,
# )

# associator = pyocto.OctoAssociator.from_area(
#     lat=(57, 61),
#     lon=(-155, -149),
#     zlim=(0, 200),
#     time_before=300,
#     velocity_model=velocity_model,
#     n_picks=20,
#     n_p_picks=2,
#     n_s_picks=10,
#     n_p_and_s_picks=2,
# )

associator = pyocto.OctoAssociator.from_area(
    lat=(57, 61),
    lon=(-155, -150),
    zlim=(0, 200),
    time_before=300,
    velocity_model=velocity_model,
    n_picks=10,
    n_p_picks=3,
    n_s_picks=5,
    n_p_and_s_picks=3,
)
    
associator.transform_stations(stations_table)
# stations_table.to_csv(filepath + 'stations_table.csv', index=False)
stations_table.head()


### Associate!

In [ ]:
# Association 
events, assignments = associator.associate(picks, stations_table)

associator.transform_events(events)
events['time'] = events['time'].apply(datetime.fromtimestamp)

all_pick_assignments = pd.merge(events, assignments, left_on="idx", right_on="event_idx", suffixes=("", "_pick"))

events.to_csv(filepath + 'events_detect_octo_staonly.csv', index=False)
all_pick_assignments.to_csv(filepath + 'all_pick_assignments_staonly.csv', index=False)

In [ ]:
### Save the results
events = pd.read_csv(filepath + 'events_detect_octo_staonly.csv')
all_pick_assignments = pd.read_csv(filepath + 'all_pick_assignments_staonly.csv')
print(len(events), len(all_pick_assignments))

### Plot
grid = pygmt.datasets.load_earth_relief(resolution="30s", region=[-155, -150, 57.5, 61])
fig = pygmt.Figure()
pygmt.config(FONT_LABEL="15p,0", 
             FONT_ANNOT_PRIMARY="15p,0",
             FONT_ANNOT_SECONDARY="15p,0", 
             MAP_FRAME_TYPE="plain")
pygmt.makecpt(cmap="terra", series=[-7000, 3000])
shade = pygmt.grdgradient(grid=grid, azimuth="0/300", normalize="e1")
fig.grdimage(grid=grid,shading=shade,projection="M10c",frame="a1",cmap=True)

pygmt.makecpt(cmap="hot", series=[0, 150])
fig.plot(
    x=events["longitude"].values,
    y=events["latitude"].values,
    size=(events["picks"].values + 20) * 0.005,
    fill=events['depth'].values,
    cmap=True,
    style="cc",
    pen="black",
)
fig.colorbar(position="JBC+w10c/0.15c+h", frame="xa50f10+levent depth (km)")
fig.show()

## 2. Catalog with Initial DAS picks

### DAS cable and initial picks -- save as the format same as stations

In [ ]:
### Cable coordinates from Ethan Williams
### We only use Ch. 500-5000
kkfls = pd.read_csv('cable_geometry/KKFLS_coords.xycz',header=None,names=['lon','lat','cha','dep'],delim_whitespace=True)
terra = pd.read_csv('cable_geometry/TERRA_coords.xycz',header=None,names=['lon','lat','cha','dep'],delim_whitespace=True)

### picks
filepaths = ['/fd1/QibinShi_data/akdas/qibin_data/plots_test_picking_dec_ch4500/phase_picks_0_200.hdf5',
             '/fd1/QibinShi_data/akdas/qibin_data/plots_test_picking_dec_ch4500/phase_picks_200_400.hdf5',
             '/fd1/QibinShi_data/akdas/qibin_data/plots_test_picking_dec_ch4500/phase_picks_400_555.hdf5']

raw_alldata_picks = np.concatenate([f["raw_alldata_picks"][:] for f in map(h5py.File, filepaths)])
mul_denoise_picks = np.concatenate([f["mul_denoise_picks"][:] for f in map(h5py.File, filepaths)])

### recording times
record_time_file = '/fd1/QibinShi_data/akdas/qibin_data/recording_times_smaller.csv'
df_record_time = pd.read_csv(record_time_file)
b_times = [UTCDateTime.strptime(start_t, format='decimator2_%Y-%m-%d_%H.%M.%S_UTC.h5') for start_t in df_record_time['record_time'].values]
b_times_terra = [b_terra + 0.88 for b_terra in b_times]
b_times_kkfls = [b_terra + 1.20 for b_terra in b_times]

### convert and save
thr = 0.05
ch_dsamp = 10
ch_ind = np.arange(0, 4500, ch_dsamp)
len_cat = raw_alldata_picks.shape[0]

channel_table = pd.DataFrame(columns=['id','longitude','latitude', 'elevation'])

for ch in ch_ind:

    if ch >= 2250:  # terra
        ch1 = int(ch * 2 - 4000) 
        longitude = terra.loc[ch1, 'lon']
        latitude = terra.loc[ch1, 'lat']
        elevation = terra.loc[ch1, 'dep']
        b_t = b_times_terra
    else:  # kkfls
        ch1 = 5000 - ch * 2
        longitude = kkfls.loc[ch1, 'lon']
        latitude = kkfls.loc[ch1, 'lat']
        elevation = kkfls.loc[ch1, 'dep']
        b_t = b_times_kkfls

    temp = pd.DataFrame(data={'id': 'das'+str(ch),
                              'longitude': longitude,
                              'latitude': latitude, 
                              'elevation': elevation},index=[0])
    channel_table = pd.concat([channel_table,temp],ignore_index=True)

    p_raw = [b_t[i] + raw_alldata_picks[i, ch, 0, 0] if raw_alldata_picks[i, ch, 0, 1] > thr else np.nan for i in range(len_cat)]
    s_raw = [b_t[i] + raw_alldata_picks[i, ch, 1, 0] if raw_alldata_picks[i, ch, 1, 1] > thr else np.nan for i in range(len_cat)]
    p_den = [b_t[i] + mul_denoise_picks[i, ch, 0, 0] if mul_denoise_picks[i, ch, 0, 1] > thr else np.nan for i in range(len_cat)]
    s_den = [b_t[i] + mul_denoise_picks[i, ch, 1, 0] if mul_denoise_picks[i, ch, 1, 1] > thr else np.nan for i in range(len_cat)]
    
    df_raw = pd.DataFrame({
        'event_id': [' '] * len_cat,
        'source_type': [' '] * len_cat,
        'station_network_code': ['CIDAS'] * len_cat,
        'station_channel_code': [' '] * len_cat,
        'station_code': ['das'+str(ch)] * len_cat,
        'station_location_code': [' '] * len_cat,
        'station_latitude_deg': [latitude] * len_cat,
        'station_longitude_deg': [longitude] * len_cat,
        'station_elevation_m': [elevation] * len_cat,
        'trace_name': [' '] * len_cat,
        'trace_sampling_rate_hz': [25] * len_cat,
        'trace_start_time': b_t,
        'trace_S_arrival_sample': [' '] * len_cat,
        'trace_P_arrival_sample': [' '] * len_cat,
        'trace_S_onset': [' '] * len_cat,
        'trace_P_onset': [' '] * len_cat,
        'trace_snr_db': [' '] * len_cat,
        'trace_s_arrival': s_raw,
        'trace_p_arrival': p_raw
    })

    df_deno = pd.DataFrame({
        'event_id': [' '] * len_cat,
        'source_type': [' '] * len_cat,
        'station_network_code': ['CIDAS'] * len_cat,
        'station_channel_code': [' '] * len_cat,
        'station_code': ['das'+str(ch)] * len_cat,
        'station_location_code': [' '] * len_cat,
        'station_latitude_deg': [latitude] * len_cat,
        'station_longitude_deg': [longitude] * len_cat,
        'station_elevation_m': [elevation] * len_cat,
        'trace_name': [' '] * len_cat,
        'trace_sampling_rate_hz': [25] * len_cat,
        'trace_start_time': b_t,
        'trace_S_arrival_sample': [' '] * len_cat,
        'trace_P_arrival_sample': [' '] * len_cat,
        'trace_S_onset': [' '] * len_cat,
        'trace_P_onset': [' '] * len_cat,
        'trace_snr_db': [' '] * len_cat,
        'trace_s_arrival': s_den,
        'trace_p_arrival': p_den
    })

    df_raw.to_csv(filepath+'1month/' + 'CIDAS_Ch_' + str(ch) + '_raw' + '.csv')
    df_deno.to_csv(filepath+'1month/' + 'CIDAS_Ch_' + str(ch) + '_deno' + '.csv')
    channel_table.to_csv(filepath + 'das_channel_table.csv', index=False)

#### Prepare channel and stations for association

In [ ]:
velocity_model = pyocto.VelocityModel0D(
    p_velocity=7.0,
    s_velocity=4.0,
    tolerance=0.5,
)
associator = pyocto.OctoAssociator.from_area(
    lat=(57, 61),
    lon=(-155, -149),
    zlim=(0, 200),
    time_before=300,
    velocity_model=velocity_model,
    n_picks=10,
    n_p_picks=0,
    n_s_picks=10,
    n_p_and_s_picks=0,
)

### DAS channels
channel_table = pd.read_csv(filepath + 'das_channel_table.csv')[::10]
associator.transform_stations(channel_table)

### Coastal stations
picks_sta = pd.read_csv(filepath + 'picks_octo.csv')
sta_table = pd.read_csv(filepath + 'stations_table.csv')

### Merge
stations_table = pd.concat([sta_table, channel_table], axis=0, ignore_index=True)
stations_table

#### Prepare picks for association

In [ ]:
### Merge picks
ch_ind = np.arange(0, 4500, 100)
all_csv =[]
for i in ch_ind:
    filename = filepath+'1month/CIDAS_Ch_' + str(i) + '_deno.csv'
    all_csv.append(pd.read_csv(filename, index_col=0))
df = pd.concat(all_csv, axis=0)

# change to Octo format
df.loc[df['trace_p_arrival'].notna(), 'phase'] = "P"
sta_p_time = df.loc[df['trace_p_arrival'].notna(), ['station_code','phase','trace_p_arrival']].rename(
    columns={"station_code": "station", "trace_p_arrival": "time"})

df.loc[df['trace_s_arrival'].notna(), 'phase'] = "S"
sta_s_time = df.loc[df['trace_s_arrival'].notna(), ['station_code','phase','trace_s_arrival']].rename(
    columns={"station_code": "station", "trace_s_arrival": "time"})

picks_das = pd.concat(objs = [sta_p_time,sta_s_time] , axis=0)
picks_das['time'] = picks_das['time'].apply(lambda x: (datetime.strptime(x,'%Y-%m-%dT%H:%M:%S.%fZ')).timestamp())

picks_das.to_csv(filepath + 'picks_octo_das.csv', index=False)

### Merge DAS and Stas
picks = pd.concat([picks_sta, picks_das], axis=0, ignore_index=True)
picks

### Associate!!

In [ ]:
# Association 
events, assignments = associator.associate(picks, stations_table)

associator.transform_events(events)
events['time'] = events['time'].apply(datetime.fromtimestamp)

all_pick_assignments = pd.merge(events, assignments, left_on="idx", right_on="event_idx", suffixes=("", "_pick"))

events.to_csv(filepath + 'events_detect_octo.csv', index=False)
all_pick_assignments.to_csv(filepath + 'all_pick_assignments.csv', index=False)

In [ ]:
### Save the results
events = pd.read_csv(filepath + 'events_detect_octo.csv')
all_pick_assignments = pd.read_csv(filepath + 'all_pick_assignments.csv')

### Plot
grid = pygmt.datasets.load_earth_relief(resolution="30s", region=[-155, -150, 57.5, 61])
fig = pygmt.Figure()
pygmt.config(FONT_LABEL="15p,0", FONT_ANNOT_PRIMARY="15p,0",
             FONT_ANNOT_SECONDARY="15p,0", MAP_FRAME_TYPE="plain")
pygmt.makecpt(cmap="terra", series=[-7000, 3000])

shade = pygmt.grdgradient(grid=grid, azimuth="0/300", normalize="e1")
fig.grdimage(grid=grid,shading=shade,projection="M10c",frame="a1",cmap=True)

pygmt.makecpt(cmap="hot", series=[0, 150])
fig.plot(
    x=events["longitude"].values,
    y=events["latitude"].values,
    size=(events["picks"].values + 20) * 0.005,
    fill=events['depth'].values,
    cmap=True,
    style="cc",
    pen="black",
)
fig.colorbar(position="JBC+w10c/0.15c+h", frame="xa50f10+levent depth (km)")
fig.show()

## 3. Catalog with Quality-Controlled DAS picks

In [ ]:
### Cable coordinates from Ethan Williams
### We only use Ch. 500-5000
kkfls = pd.read_csv('cable_geometry/KKFLS_coords.xycz',header=None,names=['lon','lat','cha','dep'],delim_whitespace=True)
terra = pd.read_csv('cable_geometry/TERRA_coords.xycz',header=None,names=['lon','lat','cha','dep'],delim_whitespace=True)

### picks
filepaths = ['/fd1/QibinShi_data/akdas/qibin_data/plots_test_picking_dec_ch4500/phase_picks_0_200.hdf5',
             '/fd1/QibinShi_data/akdas/qibin_data/plots_test_picking_dec_ch4500/phase_picks_200_400.hdf5',
             '/fd1/QibinShi_data/akdas/qibin_data/plots_test_picking_dec_ch4500/phase_picks_400_555.hdf5']

raw_alldata_picks = np.concatenate([f["raw_alldata_picks"][:] for f in map(h5py.File, filepaths)])
mul_denoise_picks = np.concatenate([f["mul_denoise_picks"][:] for f in map(h5py.File, filepaths)])
pred_picks = np.concatenate([f["predicted_picks"][:] for f in map(h5py.File, filepaths)])

### catalog
cat1 = read_events("/fd1/QibinShi_data/akdas/qibin_data/plots_test_picking_dec_ch4500/denoised_catalog_0_200.xml")
cat2 = read_events("/fd1/QibinShi_data/akdas/qibin_data/plots_test_picking_dec_ch4500/denoised_catalog_200_400.xml")
cat3 = read_events("/fd1/QibinShi_data/akdas/qibin_data/plots_test_picking_dec_ch4500/denoised_catalog_400_555.xml")
cat = cat1 + cat2 + cat3

### Recording time
df_record_time = pd.read_csv('/fd1/QibinShi_data/akdas/qibin_data/recording_times_smaller.csv')
b_times = [UTCDateTime.strptime(start_t, format='decimator2_%Y-%m-%d_%H.%M.%S_UTC.h5') for start_t in df_record_time['record_time'].values]
b_times_terra = [b_terra + 0.88 for b_terra in b_times]
b_times_kkfls = [b_terra + 1.20 for b_terra in b_times]
org_times = [evt.origins[0].time - b_t for evt, b_t in zip(cat, b_times)]
pred_picks += np.array(org_times)[:, np.newaxis, np.newaxis]

#### QC

In [ ]:
def fit_series(s1, s2, prob, thr=0.05, vmin=0, vmax=60):
    offsets = s1-s2
    ind = np.where(np.logical_and(np.logical_and(np.logical_and(vmin<s1, s1<vmax), prob > thr), np.fabs(offsets) < 3.0))[0]
    ind_rest = np.setdiff1d(np.arange(len(s1)), ind)

    s1[ind_rest] = np.nan

    return s1, ind, ind_rest

qc_picks = np.zeros_like(mul_denoise_picks)

for i in tqdm(np.arange(len(raw_alldata_picks))):
    
    qc_picks[i, :, 1, 0], ind1, ind2 = fit_series(
        mul_denoise_picks[i,:,1,0], pred_picks[i,:,1], raw_alldata_picks[i,:,1,1], thr=0.05, vmin=5, vmax=55)
    
    qc_picks[i, :, 0, 0], ind3, ind4 = fit_series(
        mul_denoise_picks[i,:,0,0], pred_picks[i,:,0], raw_alldata_picks[i,:,0,1], thr=0.1, vmin=5, vmax=55)

#### Merge qc picked with DAS locations

In [ ]:
ch_dsamp = 10
ch_ind = np.arange(0, 4500, ch_dsamp)
len_cat = qc_picks.shape[0]

for ch in ch_ind:

    if ch >= 2250:  # terra
        ch1 = int(ch * 2 - 4000) 
        longitude = terra.loc[ch1, 'lon']
        latitude = terra.loc[ch1, 'lat']
        elevation = terra.loc[ch1, 'dep']
        b_t = b_times_terra
    else:  # kkfls
        ch1 = 5000 - ch * 2
        longitude = kkfls.loc[ch1, 'lon']
        latitude = kkfls.loc[ch1, 'lat']
        elevation = kkfls.loc[ch1, 'dep']
        b_t = b_times_kkfls

    p_den = [b_t[i] + qc_picks[i, ch, 0, 0] if qc_picks[i, ch, 0, 0] == qc_picks[i, ch, 0, 0] else np.nan for i in range(len_cat)]
    s_den = [b_t[i] + qc_picks[i, ch, 1, 0] if qc_picks[i, ch, 1, 0] == qc_picks[i, ch, 1, 0] else np.nan for i in range(len_cat)]

    df_deno = pd.DataFrame({
        'event_id': [' '] * len_cat,
        'source_type': [' '] * len_cat,
        'station_network_code': ['CIDAS'] * len_cat,
        'station_channel_code': [' '] * len_cat,
        'station_code': ['das'+str(ch)] * len_cat,
        'station_location_code': [' '] * len_cat,
        'station_latitude_deg': [latitude] * len_cat,
        'station_longitude_deg': [longitude] * len_cat,
        'station_elevation_m': [elevation] * len_cat,
        'trace_name': [' '] * len_cat,
        'trace_sampling_rate_hz': [25] * len_cat,
        'trace_start_time': b_t,
        'trace_S_arrival_sample': [' '] * len_cat,
        'trace_P_arrival_sample': [' '] * len_cat,
        'trace_S_onset': [' '] * len_cat,
        'trace_P_onset': [' '] * len_cat,
        'trace_snr_db': [' '] * len_cat,
        'trace_s_arrival': s_den,
        'trace_p_arrival': p_den
    })

    df_deno.to_csv(filepath+'1month/' + 'CIDAS_Ch_' + str(ch) + '_deno_qc' + '.csv')

### Prepare stations, channel and picks for association

In [ ]:
### Initialize Octo
velocity_model = pyocto.VelocityModel0D(
    p_velocity=7.0,
    s_velocity=4.0,
    tolerance=2.0,
)
associator = pyocto.OctoAssociator.from_area(
    lat=(57, 61),
    lon=(-155, -150),
    zlim=(0, 200),
    time_before=300,
    velocity_model=velocity_model,
    n_picks=10,
    n_p_picks=3,
    n_s_picks=5,
    n_p_and_s_picks=3,
)

### DAS channel table
channel_table = pd.read_csv(filepath + 'das_channel_table.csv')[::10]
associator.transform_stations(channel_table)

### DAS picks
ch_ind = np.arange(0, 4500, 100)
all_csv =[]
for i in ch_ind:
    filename = filepath+'1month/CIDAS_Ch_' + str(i) + '_deno_qc.csv'
    all_csv.append(pd.read_csv(filename, index_col=0))
df = pd.concat(all_csv, axis=0)

# change to Octo format
df.loc[df['trace_p_arrival'].notna(), 'phase'] = "P"
sta_p_time = df.loc[df['trace_p_arrival'].notna(), ['station_code','phase','trace_p_arrival']].rename(
    columns={"station_code": "station", "trace_p_arrival": "time"})

df.loc[df['trace_s_arrival'].notna(), 'phase'] = "S"
sta_s_time = df.loc[df['trace_s_arrival'].notna(), ['station_code','phase','trace_s_arrival']].rename(
    columns={"station_code": "station", "trace_s_arrival": "time"})

picks_das = pd.concat(objs = [sta_p_time,sta_s_time] , axis=0)
picks_das['time'] = picks_das['time'].apply(lambda x: (datetime.strptime(x,'%Y-%m-%dT%H:%M:%S.%fZ')).timestamp())

picks_das.to_csv(filepath + 'picks_octo_das_qc.csv', index=False)


### Coastal data
picks_sta = pd.read_csv(filepath + 'picks_octo.csv')
sta_table = pd.read_csv(filepath + 'stations_table.csv')

### Merge DAS and Sta
picks = pd.concat([picks_sta, picks_das], axis=0, ignore_index=True)
stations_table = pd.concat([sta_table, channel_table], axis=0, ignore_index=True)

### Associate !!!

In [ ]:
# Association 
events, assignments = associator.associate(picks, stations_table)

associator.transform_events(events)
events['time'] = events['time'].apply(datetime.fromtimestamp)

all_pick_assignments = pd.merge(events, assignments, left_on="idx", right_on="event_idx", suffixes=("", "_pick"))

events.to_csv(filepath + 'events_detect_octo_qc1.csv', index=False)
all_pick_assignments.to_csv(filepath + 'all_pick_assignments_qc1.csv', index=False)

In [ ]:
### Save the results
events = pd.read_csv(filepath + 'events_detect_octo_qc1.csv')
all_pick_assignments = pd.read_csv(filepath + 'all_pick_assignments_qc1.csv')

### Plot
grid = pygmt.datasets.load_earth_relief(resolution="30s", region=[-155, -150, 57.5, 61])
fig = pygmt.Figure()
pygmt.config(FONT_LABEL="15p,0", FONT_ANNOT_PRIMARY="15p,0",
             FONT_ANNOT_SECONDARY="15p,0", MAP_FRAME_TYPE="plain")
pygmt.makecpt(cmap="terra", series=[-7000, 3000])

shade = pygmt.grdgradient(grid=grid, azimuth="0/300", normalize="e1")
fig.grdimage(grid=grid,shading=shade,projection="M10c",frame="a1",cmap=True)

pygmt.makecpt(cmap="hot", series=[0, 150])
fig.plot(
    x=events["longitude"].values,
    y=events["latitude"].values,
    size=(events["picks"].values + 20) * 0.005,
    fill=events['depth'].values,
    cmap=True,
    style="cc",
    pen="black",
)
fig.colorbar(position="JBC+w10c/0.15c+h", frame="xa50f10+levent depth (km)")
fig.show()